In [1]:
# Standard library and third party packages used by this notebook
import json
import os
from tempfile import TemporaryDirectory

import astropy.units as u
import numpy as np
import yaml

# For demonstration  plots
#from bokeh.charts import Scatter
from bokeh.models import Range1d, Span
from bokeh.layouts import row
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
import pandas

# Load Bokeh
output_notebook()

Loading BokehJS ...

In [2]:
import lsst.verify

In [3]:
demo1_metrics_yaml = """
ZeropointRMS:
  unit: mmag
  description: >
    Photometric calibration RMS.
  reference:
    url: https://example.com/PhotRMS
  tags:
    - photometry
    - demo

Completeness:
  unit: mag
  description: >
    Magnitude of the catalog's 50% completeness limit.
  reference:
    url: https://example.com/Complete
  tags:
    - photometry
    - demo
"""

In [4]:
with TemporaryDirectory() as temp_dir:
    demo1_metrics_path = os.path.join(temp_dir, 'demo1.yaml')
    with open(demo1_metrics_path, mode='w') as f:
        f.write(demo1_metrics_yaml)
    demo_metrics = lsst.verify.MetricSet.load_single_package(demo1_metrics_path)
demo_metrics

Name,Description,Units,Reference,Tags
str18,str50,str15,str28,str16
demo1.Completeness,Magnitude of the catalog's 50% completeness limit.,$\mathrm{mag}$,https://example.com/Complete,"demo, photometry"
demo1.ZeropointRMS,Photometric calibration RMS.,$\mathrm{mmag}$,https://example.com/PhotRMS,"demo, photometry"


In [5]:
zeropointrms_specs_yaml = """
---
name: "minimum"
metric: "ZeropointRMS"
threshold:
  operator: "<="
  unit: "mmag"
  value: 20.0
tags:
  - "minimum"

---
name: "minimum_megacam_r"
base: ["ZeropointRMS.minimum"]
threshold:
  value: 15.0
metadata_query:
  camera: "megacam"
  filter_name: "r"
"""

with TemporaryDirectory() as temp_dir:
    # Write YAML to disk, emulating the verify_metrics package for this demo
    specs_dirname = os.path.join(temp_dir, 'demo1')
    os.makedirs(specs_dirname)
    demo1_specs_path = os.path.join(specs_dirname, 'zeropointRMS.yaml')
    with open(demo1_specs_path, mode='w') as f:
        f.write(zeropointrms_specs_yaml)

    # Parse the YAML into a set of Specification objects
    demo1_specs = lsst.verify.SpecificationSet.load_single_package(specs_dirname)

demo1_specs

Name,Test,Tags
str36,str27,str7
demo1.ZeropointRMS.minimum,$x$ <= 20.0 $\mathrm{mmag}$,minimum
demo1.ZeropointRMS.minimum_megacam_r,$x$ <= 15.0 $\mathrm{mmag}$,minimum


In [6]:
demo1_specs_yaml = """
# Partials that define metadata queries
# for pipeline execution contexts with
# MegaCam r and u-band data, or HSC r-band.

---
id: "megacam-r"
metadata_query:
  camera: "megacam"
  filter_name: "r"

---
id: "megacam-u"
metadata_query:
  camera: "megacam"
  filter_name: "u"

---
id: "hsc-r"
metadata_query:
  camera: "hsc"
  filter_name: "r"

# We'll also write partials for each metric,
# that set up the basic test. Alternatively
# we could create full specifications to
# inherit from for each camera.

---
id: "ZeropointRMS"
metric: "demo1.ZeropointRMS"
threshold:
  operator: "<="
  unit: "mmag"

---
id: "Completeness"
metric: "demo1.Completeness"
threshold:
  operator: ">="
  unit: "mag"

# Partials to tag specifications as
# "minimum" requirements or "stretch
# goals"
---
id: "tag-minimum"
tags:
  - "minimum"

---
id: "tag-stretch"
tags:
  - "stretch"

# ZeropointRMS specifications
# tailored for each camera, in
# minimum and stretch goal variants.

---
name: "minimum_megacam_r"
base: ["#ZeropointRMS", "#megacam-r", "#tag-minimum"]
threshold:
  value: 15.0

---
name: "stretch_megacam_r"
base: ["#ZeropointRMS", "#megacam-r", "#tag-stretch"]
threshold:
  value: 10.0

---
name: "minimum_megacam_u"
base: ["#ZeropointRMS", "#megacam-u", "#tag-minimum"]
threshold:
  value: 30.0

---
name: "stretch_megacam_u"
base: ["#ZeropointRMS", "#megacam-u", "#tag-stretch"]
threshold:
  value: 20.0

---
name: "minimum_hsc_r"
base: ["#ZeropointRMS", "#hsc-r", "#tag-minimum"]
threshold:
  value: 12.0

---
name: "stretch_hsc_r"
base: ["#ZeropointRMS", "#hsc-r", "#tag-stretch"]
threshold:
  value: 6.0

# Competeness specifications,
# tailored for each camera in
# minimum and stretch goal variants

---
name: "minimum_megacam_r"
base: ["#Completeness", "#megacam-r", "#tag-minimum"]
threshold:
  value: 24.0

---
name: "stretch_megacam_r"
base: ["#Completeness", "#megacam-r", "#tag-stretch"]
threshold:
  value: 26.0

---
name: "minimum_megacam_u"
base: ["#Completeness", "#megacam-u", "#tag-minimum"]
threshold:
  value: 20.0

---
name: "stretch_megacam_u"
base: ["#Completeness", "#megacam-u", "#tag-stretch"]
threshold:
  value: 24.0

---
name: "minimum_hsc_r"
base: ["#Completeness", "#hsc-r", "#tag-minimum"]
threshold:
  value: 20.0

---
name: "stretch_hsc_r"
base: ["#Completeness", "#hsc-r", "#tag-stretch"]
threshold:
  value: 28.0
"""

with TemporaryDirectory() as temp_dir:
    # Write YAML to disk, emulating the verify_metrics package for this demo
    specs_dirname = os.path.join(temp_dir, 'demo1')
    os.makedirs(specs_dirname)
    demo1_specs_path = os.path.join(specs_dirname, 'demo1.yaml')
    with open(demo1_specs_path, mode='w') as f:
        f.write(demo1_specs_yaml)

    # Parse the YAML into a set of Specification objects
    demo_specs = lsst.verify.SpecificationSet.load_single_package(specs_dirname)

demo_specs

Name,Test,Tags
str36,str27,str7
demo1.Completeness.minimum_hsc_r,$x$ >= 20.0 $\mathrm{mag}$,minimum
demo1.Completeness.minimum_megacam_r,$x$ >= 24.0 $\mathrm{mag}$,minimum
demo1.Completeness.minimum_megacam_u,$x$ >= 20.0 $\mathrm{mag}$,minimum
demo1.Completeness.stretch_hsc_r,$x$ >= 28.0 $\mathrm{mag}$,stretch
demo1.Completeness.stretch_megacam_r,$x$ >= 26.0 $\mathrm{mag}$,stretch
demo1.Completeness.stretch_megacam_u,$x$ >= 24.0 $\mathrm{mag}$,stretch
demo1.ZeropointRMS.minimum_hsc_r,$x$ <= 12.0 $\mathrm{mmag}$,minimum
demo1.ZeropointRMS.minimum_megacam_r,$x$ <= 15.0 $\mathrm{mmag}$,minimum
demo1.ZeropointRMS.minimum_megacam_u,$x$ <= 30.0 $\mathrm{mmag}$,minimum


In [7]:
sourcecount_metric = lsst.verify.Metric(
    'demo2.SourceCount',
    "Number of matched sources.",
    unit=u.dimensionless_unscaled,
    tags=['demo'])
demo_metrics.insert(sourcecount_metric)

print(demo_metrics['demo2.SourceCount'])

demo2.SourceCount (dimensionless_unscaled): Number of matched sources.


In [8]:
sourcecount_minimum_spec = lsst.verify.ThresholdSpecification(
    'demo2.SourceCount.minimum_cfht_r',
    250 * u.dimensionless_unscaled,
    '>=',
    tags=['minimum'],
    metadata_query={'camera': 'megacam', 'filter_name': 'r'}
)
demo_specs.insert(sourcecount_minimum_spec)

sourcecount_stretch_spec = lsst.verify.ThresholdSpecification(
    'demo2.SourceCount.stretch_cfht_r',
    500 * u.dimensionless_unscaled,
    '>=',
    tags=['stretch'],
    metadata_query={'camera': 'megacam', 'filter_name': 'r'}
)
demo_specs.insert(sourcecount_stretch_spec)

In [9]:
demo_metrics

Name,Description,Units,Reference,Tags
str18,str50,str15,str28,str16
demo1.Completeness,Magnitude of the catalog's 50% completeness limit.,$\mathrm{mag}$,https://example.com/Complete,"demo, photometry"
demo1.ZeropointRMS,Photometric calibration RMS.,$\mathrm{mmag}$,https://example.com/PhotRMS,"demo, photometry"
demo2.SourceCount,Number of matched sources.,$\mathrm{}$,,demo


In [10]:
demo_specs

Name,Test,Tags
str36,str27,str7
demo1.Completeness.minimum_hsc_r,$x$ >= 20.0 $\mathrm{mag}$,minimum
demo1.Completeness.minimum_megacam_r,$x$ >= 24.0 $\mathrm{mag}$,minimum
demo1.Completeness.minimum_megacam_u,$x$ >= 20.0 $\mathrm{mag}$,minimum
demo1.Completeness.stretch_hsc_r,$x$ >= 28.0 $\mathrm{mag}$,stretch
demo1.Completeness.stretch_megacam_r,$x$ >= 26.0 $\mathrm{mag}$,stretch
demo1.Completeness.stretch_megacam_u,$x$ >= 24.0 $\mathrm{mag}$,stretch
demo1.ZeropointRMS.minimum_hsc_r,$x$ <= 12.0 $\mathrm{mmag}$,minimum
demo1.ZeropointRMS.minimum_megacam_r,$x$ <= 15.0 $\mathrm{mmag}$,minimum
demo1.ZeropointRMS.minimum_megacam_u,$x$ <= 30.0 $\mathrm{mmag}$,minimum


In [11]:
catalog_mags = np.random.uniform(18, 26, size=100)*u.mag
obs_mags = catalog_mags - 25*u.mag + np.random.normal(scale=12.0, size=100)*u.mmag

In [12]:
zp = np.median(catalog_mags - obs_mags)

In [13]:
zp_rms = np.std(catalog_mags - obs_mags)

In [14]:
zp_meas = lsst.verify.Measurement('demo1.ZeropointRMS', zp_rms)

In [15]:
zp_meas.extras['zp'] = lsst.verify.Datum(zp, label="m_0",
                                         description="Estimated zeropoint.")
zp_meas.extras['catalog_mags'] = lsst.verify.Datum(catalog_mags, label="m_cat",
                                                   description="Catalog magnitudes.")
zp_meas.extras['obs_mags'] = lsst.verify.Datum(obs_mags, label="m_obs",
                                               description="Instrument magnitudes.")

In [16]:
zp_meas.notes['estimator'] = 'numpy.std'

In [17]:
# Here's a mock dataset
mag_grid = np.linspace(22, 28, num=50, endpoint=True)
c_percent = 1. / np.cosh((mag_grid - mag_grid.min()) / 2.) * 100.

# Make the measurement
completeness_mag = np.interp(50.0, c_percent[::-1], mag_grid[::-1]) * u.mag

# Package the measurement
completeness_meas = lsst.verify.Measurement(
    'demo1.Completeness',
    completeness_mag,
)
completeness_meas.extras['mag_grid'] = lsst.verify.Datum(
    mag_grid*u.mag,
    label="m",
    description="Magnitude")
completeness_meas.extras['c_frac'] = lsst.verify.Datum(
    c_percent*u.percent,
    label="C",
    description="Photometric catalog completeness.")

In [18]:
job = lsst.verify.Job.load_metrics_package()

In [19]:
job.metrics.update(demo_metrics)
job.specs.update(demo_specs)

In [20]:
job.measurements.insert(zp_meas)
job.measurements.insert(completeness_meas)

In [21]:
job.meta.update({'camera': 'megacam', 'filter_name': 'r'})

In [22]:
print(job.meta)

{
    "camera": "megacam",
    "demo1.ZeropointRMS.estimator": "numpy.std",
    "filter_name": "r"
}


In [23]:
job.write('demo1.verify.json')

In [24]:
demo2_measurements = {}

In [25]:
demo2_measurements['demo2.SourceCount'] = 350*u.dimensionless_unscaled

In [26]:
lsst.verify.output_quantities('demo2', demo2_measurements)

'demo2.verify.json'

In [28]:
%%bash
export DYLD_LIBRARY_PATH=$LSST_LIBRARY_PATH
dispatch_verify.py --test --ignore-lsstsw --write demo.verify.json demo1.verify.json demo2.verify.json

verify.bin.dispatchverify.main INFO: Loading demo1.verify.json
verify.bin.dispatchverify.main INFO: Loading demo2.verify.json
verify.bin.dispatchverify.main INFO: Merging verification Job JSON.
verify.bin.dispatchverify.main INFO: Refreshing metric definitions from verify_metrics
verify.bin.dispatchverify.main INFO: Writing Job JSON to demo.verify.json.


In [29]:
with open('demo.verify.json') as f:
    job = lsst.verify.Job.deserialize(**json.load(f))

In [31]:
job.report().show()

Status,Specification,Measurement,Test,Metric Tags,Spec. Tags
✅,demo1.Completeness.minimum_megacam_r,24.6 $\mathrm{mag}$,$x$ >= 24.0 $\mathrm{mag}$,"demo, photometry",minimum
❌,demo1.Completeness.stretch_megacam_r,24.6 $\mathrm{mag}$,$x$ >= 26.0 $\mathrm{mag}$,"demo, photometry",stretch
✅,demo1.ZeropointRMS.minimum_megacam_r,13.3 $\mathrm{mmag}$,$x$ <= 15.0 $\mathrm{mmag}$,"demo, photometry",minimum
❌,demo1.ZeropointRMS.stretch_megacam_r,13.3 $\mathrm{mmag}$,$x$ <= 10.0 $\mathrm{mmag}$,"demo, photometry",stretch
✅,demo2.SourceCount.minimum_cfht_r,350.0 $\mathrm{}$,$x$ >= 250.0 $\mathrm{}$,demo,minimum
❌,demo2.SourceCount.stretch_cfht_r,350.0 $\mathrm{}$,$x$ >= 500.0 $\mathrm{}$,demo,stretch


In [32]:
print(job.meta)

{
    "camera": "megacam",
    "demo1.ZeropointRMS.estimator": "numpy.std",
    "filter_name": "r",
    "packages": {}
}


In [34]:
job.report(name='demo1').show()

Status,Specification,Measurement,Test,Metric Tags,Spec. Tags
✅,demo1.Completeness.minimum_megacam_r,24.6 $\mathrm{mag}$,$x$ >= 24.0 $\mathrm{mag}$,"demo, photometry",minimum
❌,demo1.Completeness.stretch_megacam_r,24.6 $\mathrm{mag}$,$x$ >= 26.0 $\mathrm{mag}$,"demo, photometry",stretch
✅,demo1.ZeropointRMS.minimum_megacam_r,13.3 $\mathrm{mmag}$,$x$ <= 15.0 $\mathrm{mmag}$,"demo, photometry",minimum
❌,demo1.ZeropointRMS.stretch_megacam_r,13.3 $\mathrm{mmag}$,$x$ <= 10.0 $\mathrm{mmag}$,"demo, photometry",stretch


In [35]:
job.report(name='demo1.ZeropointRMS').show()

Status,Specification,Measurement,Test,Metric Tags,Spec. Tags
✅,demo1.ZeropointRMS.minimum_megacam_r,13.3 $\mathrm{mmag}$,$x$ <= 15.0 $\mathrm{mmag}$,"demo, photometry",minimum
❌,demo1.ZeropointRMS.stretch_megacam_r,13.3 $\mathrm{mmag}$,$x$ <= 10.0 $\mathrm{mmag}$,"demo, photometry",stretch


In [36]:
job.specs['demo1.ZeropointRMS.minimum_megacam_r'].tags

{'minimum'}

In [38]:
job.report(spec_tags=['minimum']).show()

Status,Specification,Measurement,Test,Metric Tags,Spec. Tags
✅,demo1.Completeness.minimum_megacam_r,24.6 $\mathrm{mag}$,$x$ >= 24.0 $\mathrm{mag}$,"demo, photometry",minimum
✅,demo1.ZeropointRMS.minimum_megacam_r,13.3 $\mathrm{mmag}$,$x$ <= 15.0 $\mathrm{mmag}$,"demo, photometry",minimum
✅,demo2.SourceCount.minimum_cfht_r,350.0 $\mathrm{}$,$x$ >= 250.0 $\mathrm{}$,demo,minimum


In [39]:
job.report(spec_tags=['minimum', 'stretch']).show()

Status,Specification,Measurement,Test,Metric Tags,Spec. Tags


In [40]:
job.report(name='demo1', spec_tags=['minimum']).show()

Status,Specification,Measurement,Test,Metric Tags,Spec. Tags
✅,demo1.Completeness.minimum_megacam_r,24.6 $\mathrm{mag}$,$x$ >= 24.0 $\mathrm{mag}$,"demo, photometry",minimum
✅,demo1.ZeropointRMS.minimum_megacam_r,13.3 $\mathrm{mmag}$,$x$ <= 15.0 $\mathrm{mmag}$,"demo, photometry",minimum


In [41]:
m = job.measurements['demo1.ZeropointRMS']

In [43]:
list(m.extras.keys())

['zp', 'catalog_mags', 'obs_mags']

In [44]:
df = pandas.DataFrame({"obs_mags": m.extras['obs_mags'].quantity,
                       "catalog_mags": m.extras['catalog_mags'].quantity,
                       "delta_mags": m.extras['catalog_mags'].quantity - m.extras['obs_mags'].quantity})

In [55]:
# Scatter plot of observed vs. catalog stellar photometry
p = figure(title="Zeropoint stellar sample",
           x_axis_label="{0.label} [{0.unit}]".format(m.extras['obs_mags']),
           y_axis_label="{0.label} [{0.unit}]".format(m.extras['catalog_mags']),
           plot_width=350, plot_height=350)
p.circle(df['obs_mags'], df['catalog_mags'], color='red', fill_alpha=0.5, size=10)

GlyphRenderer(id='f796f1ab-ac73-44f7-af57-6034553bbc43', ...)

In [56]:
# Histogram of zeropoint estimates from individual matched stars.
# We're not using the Histogram Bokeh chart for some extra control.
hist_counts, hist_edges = np.histogram(df['delta_mags'], bins=10)
h = figure(tools="xpan, xwheel_zoom, reset",
           active_scroll="xwheel_zoom",
           y_range=(0, hist_counts.max()+2),
           y_axis_label="Count",
           x_axis_label="{0.label} - {1.label} [{0.unit}]".format(m.extras['obs_mags'], m.extras['catalog_mags']),
           plot_width=350, plot_height=350)
# Draw histogram edges on the figure
h.quad(bottom=0,
       left=hist_edges[:-1],
       right=hist_edges[1:],
       top=hist_counts,
       color="lightblue",
       line_color="#3A5785")
# Line at zeropoint estimate
span = Span(location=m.extras['zp'].quantity.value,
            dimension='height', line_color="black",
            line_dash='dashed', line_width=3)
h.add_layout(span)

In [57]:
# Plot side-by-side
show(row(p, h), notebook_handle=True)